In [1]:
import logging

import probtorch
import torch

import combinators
import filtering
import hmm
import utils

logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [2]:
generative = combinators.BroadcastingTrace(1)

In [3]:
hmm_params = utils.vardict({
    'mu': {
        'loc': torch.arange(5, dtype=torch.float) * 2,
        'scale': torch.ones(5) * 0.25,
    },
    'sigma': {
        'loc': torch.ones(5),
        'scale': torch.ones(5) * 0.25,
    }
})
for k in range(6):
    hmm_params['Pi_%d' % k] = {'concentration': torch.ones(5)}

In [4]:
init_hmm = combinators.Model(f=hmm.init_hmm, hyper=hmm_params)

In [5]:
z0, mu, sigma, pi, pi0 = init_hmm(trace=generative)

In [6]:
hmm_step = combinators.Model(f=hmm.hmm_step)
hmm_run = combinators.Model.sequence(hmm_step, 50, z0, mu, sigma, pi, pi0)

In [7]:
z_last, mu, sigma, pi, pi0 = hmm_run(trace=generative)

In [8]:
data = generative.unwrap(lambda k, rv: 'X_' in k)

In [9]:
num_particles = 100

In [10]:
inference = combinators.ConditionedTrace(100, data=data)

In [11]:
hmm_params = utils.vardict({
    'mu': {
        'loc': torch.arange(5, dtype=torch.float) * 2,
        'scale': torch.ones(5) * 0.25,
    },
    'sigma': {
        'loc': torch.ones(5),
        'scale': torch.ones(5) * 0.25,
    }
})
for k in range(6):
    hmm_params['Pi_%d' % k] = {'concentration': torch.ones(5)}
    hmm_params['Pi_%d' % k]['concentration'][1] = 1.25
    hmm_params['Pi_%d' % k]['concentration'][2] = 1.5
    hmm_params['Pi_%d' % k]['concentration'][3] = 1.25

In [12]:
init_hmm = combinators.Model(f=hmm.init_hmm, trainable=hmm_params)

In [13]:
z0, mu, sigma, pi, pi0 = init_hmm(trace=inference)

In [14]:
hmm_step = hmm.forward_backward_filter_hmm(mu, sigma, pi, pi0)
hmm_run = combinators.Model.sequence(hmm_step, 50, z0, mu, sigma, pi, pi0)

In [15]:
z_last, mu, sigma, pi, pi0 = hmm_run(trace=inference)

In [16]:
hmm_step.backward_pass(T=50)

In [17]:
hmm_step.smoothed_posterior(T=50)[0][3]

tensor([[ -12.4667,  -18.5687,   -3.7340,   -0.0345,   -4.6053],
        [  -5.7452,   -3.1509,   -0.2469,   -1.7558,  -25.2691],
        [  -8.1780,   -8.0047,   -1.4222,   -0.2768,  -14.8617],
        [  -7.8666,   -6.2653,   -0.6853,   -0.7057,  -19.0076],
        [  -6.8040,   -4.7056,   -1.1438,   -0.3986,  -19.1151],
        [  -3.9831,   -1.7918,   -0.3339,   -2.3167,  -22.7471],
        [  -0.0012,   -7.7226,   -7.2251,  -13.6344,  -70.7344],
        [  -0.0840,   -2.8412,   -3.8067,  -10.2908,  -56.9409],
        [ -14.1372,  -25.5279,   -5.5540,   -0.6548,   -0.7411],
        [  -2.3983,   -1.3184,   -0.4619,   -4.4690,  -35.2465],
        [ -13.8275,  -21.1483,   -5.0389,   -0.0936,   -2.4899],
        [  -3.9013,   -0.1152,   -2.4335,   -7.0417,  -36.6016],
        [ -15.4706,  -26.1870,   -6.9378,   -2.5724,   -0.0805],
        [  -0.0778,   -2.9024,   -3.9170,  -10.5885,  -48.2203],
        [  -0.0000,  -15.6274,  -11.7815,  -21.7626,  -85.1796],
        [   0.0000,  -48.

In [18]:
init_hmm_params = utils.vardict({
    'mu': {
        'loc': torch.rand(5) * 10,
        'scale': torch.ones(5),
    },
    'sigma': {
        'loc': torch.ones(5),
        'scale': torch.ones(5),
    }
})
for k in range(6):
    init_hmm_params['Pi_%d' % k] = {'concentration': torch.ones(5)}

In [19]:
init_hmm = combinators.Model(f=hmm.init_hmm, trainable=init_hmm_params)

In [20]:
def hmm_step_builder(z0, mu, sigma, pi, pi0):
    return hmm.forward_backward_filter_hmm(mu, sigma, pi, pi0)

In [21]:
inference, variational_params = filtering.variational_forward_backward(init_hmm, hmm_step_builder, 500, 50, data, use_cuda=False, lr=1e-1)

09/26/2018 13:00:42 Variational forward-backward ELBO=-7.77302002e+03 at epoch 1
09/26/2018 13:00:42 Variational forward-backward ELBO=-7.53793555e+03 at epoch 2
09/26/2018 13:00:42 Variational forward-backward ELBO=-9.24455078e+03 at epoch 3
09/26/2018 13:00:42 Variational forward-backward ELBO=-8.39255664e+03 at epoch 4
09/26/2018 13:00:43 Variational forward-backward ELBO=-7.17036768e+03 at epoch 5
09/26/2018 13:00:43 Variational forward-backward ELBO=-8.20640918e+03 at epoch 6
09/26/2018 13:00:43 Variational forward-backward ELBO=-7.51025244e+03 at epoch 7
09/26/2018 13:00:44 Variational forward-backward ELBO=-7.75716797e+03 at epoch 8
09/26/2018 13:00:44 Variational forward-backward ELBO=-7.20094727e+03 at epoch 9
09/26/2018 13:00:44 Variational forward-backward ELBO=-8.02260840e+03 at epoch 10
09/26/2018 13:00:45 Variational forward-backward ELBO=-8.20814941e+03 at epoch 11
09/26/2018 13:00:45 Variational forward-backward ELBO=-9.22155859e+03 at epoch 12
09/26/2018 13:00:46 Varia

In [22]:
variational_params

"{'Pi_0__concentration': 'tensor([[-3.3415, -5.1111, -4.6558,  8.3170, -3.1953]], grad_fn=<ExpandBackward>)', 'Pi_1__concentration': 'tensor([[13.3185,  3.8270, -3.4110, -4.6507, -4.1074]], grad_fn=<ExpandBackward>)', 'Pi_2__concentration': 'tensor([[-2.8528, -1.8208, -2.9462,  9.1282, -1.5584]], grad_fn=<ExpandBackward>)', 'Pi_3__concentration': 'tensor([[ 3.7468,  5.4777,  0.3969, 10.3747,  2.4040]], grad_fn=<ExpandBackward>)', 'Pi_4__concentration': 'tensor([[-3.7017,  1.2620, 10.8320,  6.2526, -3.3283]], grad_fn=<ExpandBackward>)', 'Pi_5__concentration': 'tensor([[-0.4266,  3.6779,  3.8597, -0.5801,  1.9135]], grad_fn=<ExpandBackward>)', 'mu__loc': 'tensor([[ 4.0544, -1.2674,  7.3752,  1.6402,  4.9851]], grad_fn=<ExpandBackward>)', 'mu__scale': 'tensor([[-3.4868, -3.0077, -3.6127, -2.3471,  0.4294]], grad_fn=<ExpandBackward>)', 'sigma__loc': 'tensor([[ 0.0052,  0.0124, -0.0056,  2.8782,  1.7460]], grad_fn=<ExpandBackward>)', 'sigma__scale': 'tensor([[-3.2660, -2.4816, -3.8008, -2.6